In [ ]:
DATA_DIR = "../data/dataset"

In [2]:
import glob
import os

filepaths = glob.glob(os.path.join(DATA_DIR, "*.jpg"))
print(f"Found {len(filepaths)} images")

Found 1162 images


In [3]:
from typing import Tuple


def get_filename(filepath: str) -> str:
    return os.path.basename(filepath).split('.')[0]


def separate_filename(filename: str) -> Tuple[str, int]:
    name, frame_id, *_ = os.path.basename(filename).split(' ')
    name = name.strip()
    frame_id = int(frame_id.strip('(').strip(')'))
    return name, frame_id

In [4]:
from typing import List


class AnnotationObject:
    def __init__(self, name: str) -> None:
        self.name = name
        self.dict_filepaths = {}
        self._filepaths = []

    def add(self, filepath: str) -> None:
        filename = get_filename(filepath)
        self.dict_filepaths[separate_filename(filename)[1]] = filepath
        listed = sorted(list(self.dict_filepaths.items()), key=lambda x: int(x[0]))
        self._filepaths = list([y for x, y in listed])

    @property
    def filepaths(self) -> List[str]:
        return self._filepaths

    def __repr__(self) -> str:
        return f"AnnotationObject({self.name}, {len(self.filepaths)})"

In [5]:
annotation_objects = {}
for filepath in filepaths:
    filename = get_filename(filepath)
    name, number = separate_filename(filename)
    if name not in annotation_objects:
        annotation_objects[name] = AnnotationObject(name)
    annotation_objects[name].add(filepath)

In [6]:
annotation_objects

{'9': AnnotationObject(9, 73),
 '4': AnnotationObject(4, 152),
 '5': AnnotationObject(5, 224),
 '11': AnnotationObject(11, 82),
 '8': AnnotationObject(8, 119),
 '6': AnnotationObject(6, 111),
 '7': AnnotationObject(7, 261),
 '1': AnnotationObject(1, 13),
 '3': AnnotationObject(3, 51),
 '12': AnnotationObject(12, 63),
 '2': AnnotationObject(2, 13)}

In [7]:
annotation_objects['12'].filepaths[0]

'../data/dataset/12 (1).jpg'

In [8]:
from fastdup import Fastdup
import pandas as pd


def find_unique_images(image_paths):
    # Создайте DataFrame из списка путей к изображениям
    df = pd.DataFrame(image_paths, columns=['filename'])

    # Инициализация Fastdup
    fd = Fastdup()

    # Запуск анализа на дубликаты
    results = fd.run(input_data=df)

    # Получение дубликатов
    duplicates = results['nearest_neighbors']

    # Фильтрация дубликатов с очень малым расстоянием
    duplicates = duplicates[duplicates['distance'] < 0.1] # порог расстояния может быть настроен

    # Извлечение всех путей из дубликатов
    duplicate_paths = set(duplicates['from']).union(set(duplicates['to']))

    # Нахождение уникальных изображений
    unique_images = [img for img in image_paths if img not in duplicate_paths]

    return unique_images


# Пример списка путей к изображениям
image_paths = annotation_objects['12'].filepaths

# Поиск уникальных изображений
unique_images = find_unique_images(image_paths)

# Вывод уникальных изображений
if unique_images:
    print(f"Найдено {len(unique_images)} уникальных изображений:")
    for img in unique_images:
        print(f"Уникальное изображение: {img}")
else:
    print("Уникальные изображения не найдены")

ModuleNotFoundError: No module named '_ctypes'